# Este es un ejemplo de un modelo Baseline de CRFs para resolver la tarea de NER, usando la herramienta crfsuite  referenciado en la página https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

## Uso de CoNLL2002 para  reconocimiento de entidades +CRF

El corpis de CoNLL2002 tiene especificados los archivos de los conjuntos de entrenamiento, evaluación y testeo. 

In [3]:
nltk.corpus.conll2002.fileids()

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [4]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

CPU times: user 1.53 s, sys: 68.8 ms, total: 1.6 s
Wall time: 1.61 s


In [5]:
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

## Selección de características

El modelo base sólo tiene como característica el token de la palabra.

sklearn-crfsuite y python-crfsuite soporta varios formatos de características.

In [6]:
def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

Se extraen las características para cada sentencia de entrenamiento, sent2features; extrae las características de forma de cada palabra, sent2labels, obtiene la etiqueta de la palabra y sent2tokens el token de la palabra.

In [7]:
#print(sent2features(train_sents[0])[0])
print(sent2labels(train_sents[0])[0])
print(sent2tokens(train_sents[0])[0])

B-LOC
Melbourne


Se genera el conjunto de entrenamiento y el de testeo

In [8]:
%%time
X_train = [sent2tokens(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2tokens(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 106 ms, sys: 4.9 ms, total: 111 ms
Wall time: 110 ms


## Training

To see all possible CRF parameters check its docstring. Here we are useing L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.

In [9]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 10.8 s, sys: 67.3 ms, total: 10.8 s
Wall time: 10.9 s


/Users/raulernestogutierrezdepinerezreyes/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

## Evaluation

There is much more O entities in data set, but we're more interested in other entities. To account for this we'll use averaged F1 score computed for all labels except for O. ``sklearn-crfsuite.metrics`` package provides some useful metrics for sequence classification task, including this one.

In [10]:

sentence2 = ['el', 'hombre', 'bajo','canta', 'bajó', 'el', 'Puente', 'rojo', 'en','Sao','Paulo','en','la','escalera', 
            'baja']
sentence=['Correr','en','Colombia','con', 'James', 'Rodriguez', 'Éste','gran', 'hombre', 'ganó', 
            'con', 'el', 'Real', 'Madrid','de','España', 'en','Alemania','con','Roler']
sentence1= ['Correr', 'en', 'Colombia', 'con', 'James', 'Rodriguez', 'Éste', 'gran', 'hombre', 'ganó', 
            'con', 'el', 'Real', 'Madrid', 'de', 'España', 'y', 'en', 'Alemania', 'con', 'Roler.']
          
def pos_tag(sentence):
    
    return list(zip(sentence, crf.predict([sentence])[0]))
 
print(pos_tag(sentence))  # [('I', 'PRP'), ('am', 'VBP'), ('Bob', 'NNP'), ('!', '.')]

[('Correr', 'B-PER'), ('en', 'I-PER'), ('Colombia', 'I-PER'), ('con', 'O'), ('James', 'B-PER'), ('Rodriguez', 'I-PER'), ('Éste', 'O'), ('gran', 'O'), ('hombre', 'O'), ('ganó', 'O'), ('con', 'O'), ('el', 'O'), ('Real', 'B-ORG'), ('Madrid', 'I-ORG'), ('de', 'I-ORG'), ('España', 'I-ORG'), ('en', 'I-ORG'), ('Alemania', 'I-ORG'), ('con', 'I-ORG'), ('Roler', 'I-ORG')]


In [11]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-LOC', 'B-ORG', 'B-PER', 'I-PER', 'B-MISC', 'I-ORG', 'I-LOC', 'I-MISC']

In [12]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.4543110726171501

Inspect per-class results in more detail:

In [13]:
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       B-LOC      0.572     0.268     0.365      1084
       I-LOC      0.326     0.138     0.194       325
      B-MISC      0.303     0.068     0.111       339
      I-MISC      0.316     0.127     0.182       557
       B-ORG      0.613     0.571     0.591      1400
       I-ORG      0.524     0.473     0.497      1104
       B-PER      0.602     0.525     0.561       735
       I-PER      0.624     0.708     0.663       634

   micro avg      0.561     0.419     0.479      6178
   macro avg      0.485     0.360     0.396      6178
weighted avg      0.531     0.419     0.454      6178

